# PDF to Podcast

## Prerequisites

1. Create a virtual environment and install the required packages.

    ```bash
    python -m venv venv
    ```

2. Install the required packages:

In [26]:
pip install -qU autogen pypdf langchain langchain-text-splitters langchain-core langchain-community lancedb langchain-openai python-dotenv azure-cognitiveservices-speech

Note: you may need to restart the kernel to use updated packages.


In [27]:
# Read .env file

from dotenv import load_dotenv

load_dotenv()

True

In [28]:
import os

def get_file(file_name: str):
  """Get file path

  Args:
      file_name (str): File name

  Returns:
      File path
  """
  output_folder = 'outputs'
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  return os.path.join(output_folder, file_name)

## PDF Information

In [29]:
# Set the pdf information

pdf_title = 'LoRA: Low-Rank Adaptation of Large Language Models'
pdf_url = 'https://arxiv.org/pdf/2106.09685'

In [30]:
# Define the file name without special characters

pdf_filename = pdf_title.replace(':', '').replace('-', '') + '.pdf'
print(pdf_filename)


LoRA LowRank Adaptation of Large Language Models.pdf


## Load PDF as langchain document

In [31]:
# Download PDF file

import requests

response = requests.get(pdf_url)

with open(get_file(pdf_filename), 'wb') as file:
  file.write(response.content)



In [32]:
# Create the documents from the PDF file

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(get_file(pdf_filename))
documents = loader.load()

In [33]:
# Update the metadata of all documents

for document in documents:
  document.metadata['title'] = pdf_title
  document.metadata['source'] = pdf_url
  document.metadata['description'] = ''
  document.metadata['thumbnail_url'] = ''
  document.metadata['type'] = 'pdf'

## Create embeddings

In [34]:
# Split the document in chunks of maximum 1000 characters with 200 characters overlap using langchain

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200
)
splits = text_splitter.split_documents(documents)

In [35]:
# Define the embeddings model

from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
  api_key=os.environ['OPENAI_API_KEY'],
  azure_endpoint=os.environ['OPENAI_AZURE_ENDPOINT'],
  api_version=os.environ['OPENAI_API_VERSION'],
  azure_deployment=os.environ['OPENAI_AZURE_DEPLOYMENT_EMBEDDINGS']
)

In [36]:
# Create the vector store

import lancedb
from langchain_community.vectorstores import LanceDB

db = lancedb.connect("/tmp/lancedb")

vectorstore = LanceDB.from_documents(
  documents=splits,
  embedding=azure_openai_embeddings
)

retriever = vectorstore.as_retriever()

In [37]:
# Clean up: delete the downloaded PDF file

os.remove(get_file(pdf_filename))

## Create the langchain chain to do RAG

In [38]:
# Create the prompt for the chain with embeddings and LLM

from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [39]:
# Define the LLM model

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
  azure_endpoint=os.environ['OPENAI_AZURE_ENDPOINT'],
  api_version=os.environ['OPENAI_API_VERSION'],
  azure_deployment=os.environ['OPENAI_AZURE_DEPLOYMENT'],
  temperature=0,
  top_p=1
)

In [40]:
# Define the rag chain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

## Create the outline of the podcast

In [41]:
podcast_outline_response = rag_chain.invoke({"input": "Create an outline for a podcast on LoRA."})
podcast_outline = podcast_outline_response['answer']

[2024-09-18T08:04:12Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:04:12Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:04:12Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


In [42]:
# Write the podcast outline

podcast_outline_file_name = pdf_filename.replace('.pdf', '_script.txt')

with open(get_file(podcast_outline_file_name), "w") as f:
    f.write(podcast_outline)

## Create the podcast script

In [43]:
def read_and_print_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        return content
        #print(content)

# Example usage
file_path = '/workspaces/AutoPodCaster/data/How to fine-tune a model using LoRA (step by step)_podcast_outline.txt'  # Replace with the path to your file
podcast_outline = read_and_print_file(file_path)


In [44]:
segments_outline = podcast_outline.split("####")
print(segments_outline)

['### Podcast Outline: How to Fine-Tune a Model Using LoRA (Step by Step)\n\n', " Introduction\n1. **Welcome and Introduction**\n   - Brief introduction to the podcast and today's topic.\n   - Overview of what listeners will learn about fine-tuning models using LoRA.\n\n2. **What is LoRA?**\n   - Explanation of LoRA (Low-Rank Adaptation).\n   - Benefits of using LoRA for fine-tuning models.\n   - Importance of fine-tuning large models for specific tasks.\n\n", ' Segment 1: Setting Up for Fine-Tuning\n1. **Choosing the Right Hardware**\n   - Importance of using a GPU for fine-tuning.\n   - Example of using an L4 GPU and its cost-effectiveness.\n\n2. **Loading the Base Model**\n   - Steps to load the base model into memory.\n   - Printing the number of parameters in the base model.\n\n', ' Segment 2: Understanding LoRA Configuration\n1. **Decomposing Matrices**\n   - Explanation of rank one matrices and their role in LoRA.\n   - Trade-offs between lower and higher ranks in terms of accur

In [45]:
from IPython.display import Image, display

import autogen
from autogen.coding import LocalCommandLineCodeExecutor

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST"
)

In [60]:
import json
import os

# create an AssistantAgent named "assistant"
writer = autogen.AssistantAgent(
    name="writer",
    system_message="""You are a writer of a podcast. If you get a bad review from the reviewer on a segment of a podcast, you need to rewrite that podcast segment. 
    Output the rewritten segment as a JSON with the following fields:
        - title: Title of the podcast
        - text: an array of objects with the speaker, the intonation and the text to be spoken
        Return only the json as plain text.
    "Return 'TERMINATE' when the task is done.""",
    llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

reviewer = autogen.AssistantAgent(
    name="reviewer",
    system_message="""You are a reviewer of a podcast.
    If you see questions and answers that are duplicate one, please ask to remove them. 
    "Return 'TERMINATE' when the task is done.""",
    llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    # code_execution_config={
    #     # the executor to run the generated code
    #     "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    # },
)

groupchat = autogen.GroupChat(agents=[user_proxy, reviewer, writer], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0  # temperature for sampling
    })

In [61]:
podcast_conversation = ""

for segment in range(len(segments_outline[1:])):

    # create an AssistantAgent named "assistant"
    writer = autogen.AssistantAgent(
        name="writer",
        system_message="""You are a writer of a podcast. If you get a bad review from the reviewer on a segment of a podcast, you need to rewrite that podcast segment. 
        Output the rewritten segment as a JSON with the following fields:
            - title: Title of the podcast
            - text: an array of objects with the speaker, the intonation and the text to be spoken
            Return only the json as plain text.
        "Return 'TERMINATE' when the task is done.""",
        llm_config={
            "cache_seed": 41,  # seed for caching and reproducibility
            "config_list": config_list,  # a list of OpenAI API configurations
            "temperature": 0  # temperature for sampling
        },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
    )

    reviewer = autogen.AssistantAgent(
        name="reviewer",
        system_message="""You are a reviewer of a podcast.
        If you see questions and answers that are duplicate one, please ask to remove them. 
        "Return 'TERMINATE' when the task is done.""",
        llm_config={
            "cache_seed": 41,  # seed for caching and reproducibility
            "config_list": config_list,  # a list of OpenAI API configurations
            "temperature": 0  # temperature for sampling
        },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
    )
    # create a UserProxyAgent instance named "user_proxy"
    user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        human_input_mode="NEVER",
        max_consecutive_auto_reply=10,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
        # code_execution_config={
        #     # the executor to run the generated code
        #     "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
        # },
    )

    groupchat = autogen.GroupChat(agents=[user_proxy, reviewer, writer], messages=[], max_round=12)
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={
            "cache_seed": 41,  # seed for caching and reproducibility
            "config_list": config_list,  # a list of OpenAI API configurations
            "temperature": 0  # temperature for sampling
        })
    
    # Create a prompt with the outline to get a full podcast text
    if len(podcast_conversation) == 0:
        podcast_prompt = f"""Create the first segment of a podcast text which is the introduction based on the following part of an outline:

        {segments_outline[segment]}

        This text will be used to generate the audio of the podcast. 
        There are 2 participants in the podcast: the host and the guest. 
        The host will introduce the podcast and the guest. 
        Both the host and the speaker should stick to the {segments_outline[segment]} as topic of the podcast.
        The host should follow the {segments_outline[segment]} for asking questions to the guest.
        The name of the host is Pierre and his role is to be the listener's podcast assistant. 
        The name of the guest is Marie and her role is to be the expert in the podcast topic. 
        The name of the podcast is "Advanced AI Podcast".
        

        When you thanks someone, write "Thank you" and the name of the person without a comma. For example, "Thank you Pierre".

        Output as a JSON with the following fields:
        - title: Title of the podcast
        - text: an array of objects with the speaker, the intonation and the text to be spoken
        Return only the json as plain text.
        """
    elif len(podcast_conversation) != 0 and segment!=len(segments_outline)-1:
        podcast_prompt = f"""Create a segment which is in the middle of a podcast text based on the following part of an outline:

        {segments_outline[segment]}

        This text will be used to generate the audio of the podcast. 
        There are 2 participants in the podcast: the host and the guest. 
        The host will ask questions about the {segments_outline[segment]} to the guest and the guest will answer them. 
        Both the host and the speaker should stick to the {segments_outline[segment]} as topic of the podcast.
        The host should follow the {segments_outline[segment]} for asking questions to the guest.
        The name of the host is Pierre and his role is to be the listener's podcast assistant. 
        The name of the guest is Marie and her role is to be the expert in the podcast topic.
        This is in the middle of the podcast, so don't welcome the listeners again and don't specify what this segment is about!

        Output as a JSON with the following fields:
        - text: an array of objects with the speaker, the intonation and the text to be spoken
        Return only the json as plain text.
        """
    elif segment==len(segments_outline)-1 :
        podcast_prompt = f"""Create a segment which is the closing of a podcast text based on the following part of an outline:

        {segments_outline[segment]}

        This text will be used to generate the audio of the podcast. 
        There are 2 participants in the podcast: the host and the guest.  
        The host will thank the guest and close the podcast.
        The host should follow the {segments_outline[segment]} for asking questions to the guest.
        Both the host and the speaker should stick to the {segments_outline[segment]} as topic of the podcast.
        The name of the host is Pierre and his role is to be the listener's podcast assistant. 
        The name of the guest is Marie and her role is to be the expert in the podcast topic. 

        When you thanks someone, write "Thank you" and the name of the person without a comma. For example, "Thank you Pierre".

        Output as a JSON with the following fields:
        - text: an array of objects with the speaker, the intonation and the text to be spoken
        Return only the json as plain text.
        """

    formatted_podcast_prompt = podcast_prompt.format(segments_outline[segment])

    podcast_script_response = rag_chain.invoke({"input": formatted_podcast_prompt})
    podcast_script_text = podcast_script_response['answer']
    
    chat_res = user_proxy.initiate_chat(
    recipient=manager,
    message=f"""Combine""" + podcast_conversation + " and " + podcast_script_text + f""" and evaluate if the conversation is 
    a coherent one, has a good narative, if it has a same conversation style, and a seamless transition between the two parts. 
    """,
    # message="""Combine""" + podcast_conversation + " and " + podcast_script_text + """ and evaluate if the conversation is 
    # a coherent one, has a good narative, if it has a same conversation style, and a seamless transition between the two parts. 
    # If you see questions and answers that are duplicate one, please remove them. Give a bad review and ask to rewrite!
    # """,
    summary_method="reflection_with_llm"
    )   

    podcast_conversation = podcast_conversation + podcast_script_text

    # print(podcast_script_text)
    

[2024-09-18T08:30:03Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:30:03Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:30:03Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


user_proxy (to chat_manager):

Combine and {
  "title": "How to Fine-Tune a Model Using LoRA (Step by Step)",
  "text": [
    {
      "speaker": "Pierre",
      "intonation": "enthusiastic",
      "text": "Welcome to the Advanced AI Podcast! I'm your host, Pierre, and today we have an exciting episode lined up for you. We're diving deep into the world of model fine-tuning using LoRA, or Low-Rank Adaptation. This method is revolutionizing how we adapt large language models for specific tasks, making the process more efficient and cost-effective."
    },
    {
      "speaker": "Pierre",
      "intonation": "warm",
      "text": "Joining us today is Marie, an expert in this fascinating field. Marie, thank you for being here and sharing your insights with us."
    },
    {
      "speaker": "Marie",
      "intonation": "grateful",
      "text": "Thank you Pierre. I'm thrilled to be here and discuss how LoRA can make a significant impact on model fine-tuning."
    },
    {
      "speaker": "

[2024-09-18T08:30:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:30:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:30:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


user_proxy (to chat_manager):

Combine{
  "title": "How to Fine-Tune a Model Using LoRA (Step by Step)",
  "text": [
    {
      "speaker": "Pierre",
      "intonation": "enthusiastic",
      "text": "Welcome to the Advanced AI Podcast! I'm your host, Pierre, and today we have an exciting episode lined up for you. We're diving deep into the world of model fine-tuning using LoRA, or Low-Rank Adaptation. This method is revolutionizing how we adapt large language models for specific tasks, making the process more efficient and cost-effective."
    },
    {
      "speaker": "Pierre",
      "intonation": "warm",
      "text": "Joining us today is Marie, an expert in this fascinating field. Marie, thank you for being here and sharing your insights with us."
    },
    {
      "speaker": "Marie",
      "intonation": "grateful",
      "text": "Thank you Pierre. I'm thrilled to be here and discuss how LoRA can make a significant impact on model fine-tuning."
    },
    {
      "speaker": "Pierr

[2024-09-18T08:30:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:30:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:30:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


user_proxy (to chat_manager):

Combine{
  "title": "How to Fine-Tune a Model Using LoRA (Step by Step)",
  "text": [
    {
      "speaker": "Pierre",
      "intonation": "enthusiastic",
      "text": "Welcome to the Advanced AI Podcast! I'm your host, Pierre, and today we have an exciting episode lined up for you. We're diving deep into the world of model fine-tuning using LoRA, or Low-Rank Adaptation. This method is revolutionizing how we adapt large language models for specific tasks, making the process more efficient and cost-effective."
    },
    {
      "speaker": "Pierre",
      "intonation": "warm",
      "text": "Joining us today is Marie, an expert in this fascinating field. Marie, thank you for being here and sharing your insights with us."
    },
    {
      "speaker": "Marie",
      "intonation": "grateful",
      "text": "Thank you Pierre. I'm thrilled to be here and discuss how LoRA can make a significant impact on model fine-tuning."
    },
    {
      "speaker": "Pierr

[2024-09-18T08:32:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:32:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-09-18T08:32:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


user_proxy (to chat_manager):

Combine{
  "title": "How to Fine-Tune a Model Using LoRA (Step by Step)",
  "text": [
    {
      "speaker": "Pierre",
      "intonation": "enthusiastic",
      "text": "Welcome to the Advanced AI Podcast! I'm your host, Pierre, and today we have an exciting episode lined up for you. We're diving deep into the world of model fine-tuning using LoRA, or Low-Rank Adaptation. This method is revolutionizing how we adapt large language models for specific tasks, making the process more efficient and cost-effective."
    },
    {
      "speaker": "Pierre",
      "intonation": "warm",
      "text": "Joining us today is Marie, an expert in this fascinating field. Marie, thank you for being here and sharing your insights with us."
    },
    {
      "speaker": "Marie",
      "intonation": "grateful",
      "text": "Thank you Pierre. I'm thrilled to be here and discuss how LoRA can make a significant impact on model fine-tuning."
    },
    {
      "speaker": "Pierr

KeyboardInterrupt: 

In [65]:
for segment in segments_outline:
    print(i)
    print(segments_outline[i])
    print("--------------------------")
    print(segments_outline[i-1])

ValueError: too many values to unpack (expected 2)

In [64]:
for i in range(len(segments_outline[1:])):
    print(i)
    print(segments_outline[i])
    print("--------------------------")
    print(segments_outline[i-1])

0
### Podcast Outline: How to Fine-Tune a Model Using LoRA (Step by Step)


--------------------------
 Q&A Segment (Optional)
1. **Listener Questions**
   - Answering common questions about fine-tuning models and using LoRA.
   - Providing additional tips and insights based on listener feedback.
1
 Introduction
1. **Welcome and Introduction**
   - Brief introduction to the podcast and today's topic.
   - Overview of what listeners will learn about fine-tuning models using LoRA.

2. **What is LoRA?**
   - Explanation of LoRA (Low-Rank Adaptation).
   - Benefits of using LoRA for fine-tuning models.
   - Importance of fine-tuning large models for specific tasks.


--------------------------
### Podcast Outline: How to Fine-Tune a Model Using LoRA (Step by Step)


2
 Segment 1: Setting Up for Fine-Tuning
1. **Choosing the Right Hardware**
   - Importance of using a GPU for fine-tuning.
   - Example of using an L4 GPU and its cost-effectiveness.

2. **Loading the Base Model**
   - Steps t

In [59]:
print(chat_res.chat_history[2]['content'])

{
  "title": "How to Fine-Tune a Model Using LoRA (Step by Step)",
  "text": [
    {
      "speaker": "Pierre",
      "intonation": "enthusiastic",
      "text": "Welcome to the Advanced AI Podcast! I'm your host, Pierre, your go-to podcast assistant for all things AI. Today, we have an exciting episode lined up for you. We're diving deep into the world of model fine-tuning using LoRA, or Low-Rank Adaptation. And to guide us through this fascinating topic, we have a special guest with us, Marie, an expert in the field. Welcome to the podcast, Marie!"
    },
    {
      "speaker": "Marie",
      "intonation": "warm",
      "text": "Thank you Pierre. I'm thrilled to be here and to share insights on how LoRA can make model fine-tuning more efficient and effective."
    },
    {
      "speaker": "Pierre",
      "intonation": "curious",
      "text": "Fantastic! Let's get started. Marie, could you begin by giving us a brief overview of what LoRA is and why it's important in the context of f

In [79]:
# Create a prompt with the outline to get a full podcast text

podcast_prompt = f"""Create a podcast complete text based on the following outline:

{podcast_outline}

This text will be used to generate the audio of the podcast. There are 2 participants in the podcast: the host and the guest. The host will introduce the podcast and the guest. The guest will explain the outline of the podcast. The host will ask questions to the guest and the guest will answer them. The host will thank the guest and close the podcast.
The name of the host is Pierre and his role is to be the listener's podcast assistant. The name of the guest is Marie and her role is to be the expert in the podcast topic. The name of the podcast is "Advanced AI Podcast".

When you thanks someone, write "Thank you" and the name of the person without a comma. For example, "Thank you Pierre".

Output as a JSON with the following fields:
- title: Title of the podcast
- text: an array of objects with the speaker, the intonation and the text to be spoken
Return only the json as plain text.
"""

formatted_podcast_prompt = podcast_prompt.format(podcast_outline)

In [80]:
# Generate the podcast script

podcast_script_response = rag_chain.invoke({"input": formatted_podcast_prompt})
podcast_script_text = podcast_script_response['answer']

In [81]:
# Save the podcast script

podcast_script_file_name = pdf_filename.replace('.pdf', '_script.json')

with open(get_file(podcast_script_file_name), "w") as f:
    f.write(podcast_script_text)

## Generate the podcast audio

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import json

# Creates an instance of a speech config with specified subscription key and service region.
speech_key = os.environ['AZURE_SPEECH_KEY']
service_region = os.environ['AZURE_SPEECH_REGION']

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# This is an example of SSML (Speech Synthesis Markup Language) format.
# <speak version="1.0" xmlns="https://www.w3.org/2001/10/synthesis" xml:lang="en-US">
#   <voice name="en-US-AvaMultilingualNeural">
#     When you're on the freeway, it's a good idea to use a GPS.
#   </voice>
# </speak>
# Parse the JSON response and create a SSML with en-US-GuyNeural for Pierre Voice
# and en-US-JennyNeural for Marie Voice
podcast_script_json = json.loads(str(podcast_script_text))
ssml_text = "<speak version='1.0' xmlns='https://www.w3.org/2001/10/synthesis' xml:lang='en-US'>"
for line in podcast_script_json['text']:
    speaker = line['speaker']
    text = line['text']
    if speaker == 'Pierre':
        ssml_text += f"<voice name='en-US-GuyNeural'>{text}</voice>"
    elif speaker == 'Marie':
        ssml_text += f"<voice name='en-US-JennyNeural'>{text}</voice>"
ssml_text += "</speak>"

# use the default speaker as audio output.
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

result = speech_synthesizer.speak_ssml_async(ssml_text).get()
stream = speechsdk.AudioDataStream(result)
podcast_filename = pdf_filename.replace('.pdf', '_podcast.wav')
stream.save_to_wav_file(get_file(podcast_filename))

